In [38]:
import pandas as pd
import numpy as np
import pandasql
import scipy.stats


def num_rainy_days(filename):
    '''
    This function should run a SQL query on a dataframe of
    weather data.  The SQL query should return one column and
    one row - a count of the number of days in the dataframe where
    the rain column is equal to 1 (i.e., the number of days it
    rained).      
    You might also find that interpreting numbers as integers or floats may not
    work initially.  In order to get around this issue, it may be useful to cast
    these numbers as integers.  This can be done by writing cast(column as integer).
    So for example, if we wanted to cast the maxtempi column as an integer, we would actually
    write something like where cast(maxtempi as integer) = 76, as opposed to simply 
    where maxtempi = 76.
    
    You can see the weather data that we are passing in below:
    https://www.dropbox.com/s/7sf0yqc9ykpq3w8/weather_underground.csv
    '''
    weather_data = pd.read_csv(filename)

    q = """
    select count(rain) from weather_data group by rain
    """
    
    #Execute your SQL command against the pandas frame
    rainy_days = pandasql.sqldf(q.lower(), locals())
    return rainy_days

def compare_averages(filename):
    weather_data=pd.read_csv(filename)
    
    q = """
    select ENTRIESn_hourly from weather_data where rain == 0
    """
    rrs = pandasql.sqldf(q.lower(), locals())
    
    q = """
    select ENTRIESn_hourly from weather_data where rain == 1
    """
    nrs = pandasql.sqldf(q.lower(), locals())

    rg = scipy.stats.ttest_ind(rrs['ENTRIESn_hourly'],nrs['ENTRIESn_hourly'], equal_var=False)
    if rg[1]>=0.5:
        return(True,rg)
    else:
        return(False,rg)
    
def mann_whitney_plus_means(filename):
    '''
    This function will consume the turnstile_weather dataframe containing
    our final turnstile weather data. 
    
    You will want to take the means and run the Mann Whitney U-test on the 
    ENTRIESn_hourly column in the turnstile_weather dataframe.
    
    This function should return:
        1) the mean of entries with rain
        2) the mean of entries without rain
        3) the Mann-Whitney U-statistic and p-value comparing the number of entries
           with rain and the number of entries without rain
    
    You should feel free to use scipy's Mann-Whitney implementation, and you 
    might also find it useful to use numpy's mean function.
    
    Here are the functions' documentation:
    http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html
    http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html
    '''

    weather_data=pd.read_csv(filename)
    
    wrm=weather_data[weather_data['rain']==1]['ENTRIESn_hourly']
    with_rain_mean=np.mean(wrm)
    worm=weather_data[weather_data['rain']==0]['ENTRIESn_hourly']
    without_rain_mean=np.mean(worm)
    [U,p]=scipy.stats.mannwhitneyu(wrm, worm)
    
    return with_rain_mean, without_rain_mean, U, p # leave this line for the grader
    
if __name__ == "__main__":
    filename = "improved-dataset/turnstile_weather_v2.csv"
    print num_rainy_days(filename)
    print compare_averages(filename)
    print "(mean w rain, mean w/o rain, U, p) = ", mann_whitney_plus_means(filename)

   count(rain)
0        33064
1         9585
(False, Ttest_indResult(statistic=-5.0428827476194309, pvalue=4.6414024316324798e-07))
(mean w rain, mean w/o rain, U, p) =  (2028.1960354720918, 1845.5394386644084, 153635120.5, 2.7410695712437496e-06)


In [54]:
# Linear Regression OLS

import numpy as np
import pandas
import statsmodels.api as sm

"""
In this question, you need to:
1) implement the linear_regression() procedure
2) Select features (in the predictions procedure) and make predictions.

"""

def linear_regression(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features.
    
    This can be the same code as in the lesson #3 exercise.
    """
    
    ###########################
    ### YOUR CODE GOES HERE ###
    ###########################
    features=sm.add_constant(features)
    model=sm.OLS(values,features)
    results=model.fit()
    intercept = results.params[0]
    params=results.params[1:]
    
    return intercept, params

def predictions(dataframe):
    '''
    The NYC turnstile data is stored in a pandas dataframe called weather_turnstile.
    Using the information stored in the dataframe, let's predict the ridership of
    the NYC subway using linear regression with gradient descent.
    
    Your prediction should have a R^2 value of 0.40 or better.
    You need to experiment using various input features contained in the dataframe. 
    We recommend that you don't use the EXITSn_hourly feature as an input to the 
    linear model because we cannot use it as a predictor: we cannot use exits 
    counts as a way to predict entry counts. 
    '''
    ################################ MODIFY THIS SECTION #####################################
    # Select features. You should modify this section to try different features!             #
    # I've selected rain, fog, 'hour, weekday, and UNIT (as a dummy) to start you off.       #
    # See this page for more info about dummy variables:                                     #
    # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html          #
    ##########################################################################################
    features = dataframe[['rain', 'fog', 'hour', 'weekday']]
    dummy_units = pandas.get_dummies(dataframe['UNIT'], prefix='unit')
    features = features.join(dummy_units)
    
    # Values
    values = dataframe['ENTRIESn_hourly']

    # Perform linear regression
    intercept, params = linear_regression(features, values)
    predictions = intercept + np.dot(features, params)
    
    print "intercept: ", intercept
    print "params: ", params[:4]

    return predictions

def compute_r_squared(data, predictions):
    # Given two input numpy arrays, 'data', and 'predictions,'
    # returns the coefficient of determination, R^2, for the model that produced 
    # predictions.
        
    avg_data=np.mean(data)
    r_squared = 1 - np.sum((data-predictions)**2)/np.sum((data-avg_data)**2)

    return r_squared

def plot_residuals(turnstile_weather, predictions):
    '''
    the difference between the original hourly entry data and the predicted values
    '''
    
    plt.figure()
    diff=(turnstile_weather['ENTRIESn_hourly'] - predictions)
    
    df = pd.DataFrame({'real-predicted': diff},columns=['real-predicted'])
    df.plot(kind='hist',alpha=0.5, bins=50)
    return plt

if __name__ == "__main__":
    input_filename = "improved-dataset/turnstile_weather_v2.csv"
    turnstile_master = pd.read_csv(input_filename)
    predicted_values = predictions(turnstile_master)
    r_squared = compute_r_squared(turnstile_master['ENTRIESn_hourly'], predicted_values) 
    print r_squared
    
    image = "plot_residu_hist.png"
    plt = plot_residuals(turnstile_master, predicted_values)
    plt.savefig(image)


intercept:  -103.779608371
params:  rain        57.936517
fog       -596.515322
hour       123.712526
weekday    973.276640
dtype: float64
0.481783370977


In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def entries_histogram(turnstile_weather):
    '''
    Let's plot two histograms on the same axes to show hourly
    entries when raining vs. when not raining. Here's an example on how
    to plot histograms with pandas and matplotlib:
    turnstile_weather['column_to_graph'].hist()

    You can read a bit about using matplotlib and pandas to plot histograms here:
    http://pandas.pydata.org/pandas-docs/stable/visualization.html#histograms
    
    '''
    plt.figure()
    wrm=turnstile_weather[turnstile_weather['rain']==1]['ENTRIESn_hourly']
    worm=turnstile_weather[turnstile_weather['rain']==0]['ENTRIESn_hourly']
    #print "with_rain_mean", np.mean(wrm)
    #print "without_rain_mean", np.mean(worm)
    
    df = pd.DataFrame({'w rain': wrm,
                      'wo rain':worm},
                      columns=['w rain','wo rain'])
    df.plot(kind='hist',alpha=0.5, bins=50)
    plt.title('Histogram of ENTRIESn_hourly')
    plt.xlabel('ENTRIESn_hourly')
    plt.ylabel('Frequency')
    plt.legend()

    return plt


if __name__ == "__main__":
    image = "plot_tw_hist.png"
    turnstile_weather = pd.read_csv("improved-dataset/turnstile_weather_v2.csv")
    plt = entries_histogram(turnstile_weather)
    plt.savefig(image)

In [ ]:
import pandas as pd
from ggplot import *
import datetime

def plot_weather_data(entries_Day):
    plot = ggplot(entries_Day, aes(x='Day', y='ENTRIESn_hourly')) \
    + geom_bar(aes(weight='ENTRIESn_hourly'), fill='blue',stat="bar") \
    + ggtitle('NYC Subway ridership / day of week') \
    + xlab('Day') + ylab('Entries')
    
    return plot

if __name__ == "__main__":
    image = "plot_ridership.png"
    turnstile_weather = pd.read_csv("improved-dataset/turnstile_weather_v2.csv")
    entries_DayOfMonth = turnstile_weather[['DATEn', 'ENTRIESn_hourly']].groupby('DATEn', as_index=False).sum()
    entries_DayOfMonth['Day'] = [datetime.strptime(x, '%m-%d-%y').strftime('%w %A') 
    for x in entries_DayOfMonth['DATEn']]
    entries_Day = entries_DayOfMonth[['Day', 'ENTRIESn_hourly']].groupby('Day', as_index=False).sum()
    
    gg =  plot_weather_data(entries_Day)
    #print gg
    #ggsave(gg,image)

In [59]:
from ggplot import *
import pandas as pd

def lineplot_compare(turnstile_master):

    gg = ggplot(turnstile_master,aes('DATEn','ENTRIESn_hourly', color='rain'))+geom_point()+geom_line()
    return gg


if __name__ == "__main__":
    input_filename = "improved-dataset/turnstile_weather_v2.csv"
    turnstile_master = pd.read_csv(input_filename)

    image = "plot_residu_line.png"
    gg =  lineplot_compare(turnstile_master)
    #ggsave(image, gg, width=11, height=8)